In [9]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from util import parse_ingredient_list
from itertools import chain
import re
import json
import nltk
import pycrfsuite
import spacy.lang.en

In [178]:
# load one of the datasets to look at
file_name = "recipes_raw_nosource_ar"
df = pd.read_json("data/"+file_name+".json")
df.head()

,rmK12Uau.ntP510KeImX506H6Mr6jTu,5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,clyYQv.CplpwJtjNaFGhx0VilNYqRxu,BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,kq.naD.8G19M4UU9dVvJgHtpfo.l/eC,lYrgWNn00EXblOupzM3tL0jGr9O0CB2,Fu0DgGYFUGwc0BBlN6r20o/ihOVs5bO,MBRNtqELRRuv8zJH4k7Aba2bmIc2A3C,ZPyPoMiNvgAfrKcRpH9FEYV/XsPZBsW,...,uHHb42/tuIKsmN5U6l9AD.FdVpSFxs6,IPAeN3L6rm2oughJpUhbG038k.ACJ0K,3UIhlTQFH5jyIaHN8zeKlK5V.94Kjwu,PdBxkE2gnI/.ynokkp1Hu1KLGZnGdei,SB46Udqc5Svsi70S1qRmRLv5tlg8Oca,gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
title,Slow Cooker Chicken and Dumplings,Awesome Slow Cooker Pot Roast,Brown Sugar Meatloaf,Best Chocolate Chip Cookies,Homemade Mac and Cheese Casserole,Banana Banana Bread,Chef John's Fisherman's Pie,Mom's Zucchini Bread,The Best Rolled Sugar Cookies,Singapore Chili Crabs,...,Citrus Creme Brulee,Honey Nutty Granola,Salmon en Croute,Homemade Blender Butter,Gluten Free Chicken Noodle Soup,Thai-Indian Veggie Soup,Coconut Milk-Free Panang Curry Chicken,Cooked Cold Salad,Easy Eggnog Creme Brulee,Super Power Stovetop Granola
ingredients,"[4 skinless, boneless chicken breast halves, 2...",[2 (10.75 ounce) cans condensed cream of mushr...,"[1/2 cup packed brown sugar, 1/2 cup ketchup, ...","[1 cup butter, softened, 1 cup white sugar, 1 ...","[8 ounces whole wheat rotini pasta, 3 cups fre...","[2 cups all-purpose flour, 1 teaspoon baking s...","[For potato crust:, 3 russet potatoes, peeled ...","[3 cups all-purpose flour, 1 teaspoon salt, 1 ...","[1 1/2 cups butter, softened, 2 cups white sug...","[Sauce:, 1/2 cup ketchup, 1/2 cup chicken brot...",...,"[2 oranges, juiced, lemon, juiced, 1 lime, jui...","[3 cups rolled oats, 1 1/2 cups toasted almond...","[1 cup watercress, or as desired, 1/2 lemon, z...","[2 pints heavy whipping cream, salt to taste]",[1/2 (12 ounce) box Barilla® Gluten Free Elbow...,"[2 teaspoons olive oil, 1/4 cup minced fresh g...","[2 cups light cream, 1/4 teaspoon coconut extr...","[3 tablespoons bacon grease, 2 cups shredded B...","[4 egg yolks, 1 tablespoon white sugar, 1 cup ...","[1/4 cup canola oil, 3 cups quick-cooking oats..."
instructions,"Place the chicken, butter, soup, and onion in ...","In a slow cooker, mix cream of mushroom soup, ...",Preheat oven to 350 degrees F (175 degrees C)....,Preheat oven to 350 degrees F (175 degrees C)....,Preheat oven to 350 degrees F. Line a 2-quart ...,Preheat oven to 350 degrees F (175 degrees C)....,Bring a large saucepan of salted water and to ...,Grease and flour two 8 x 4 inch pans. Preheat ...,"In a large bowl, cream together butter and sug...","Whisk ketchup, chicken broth, egg, soy sauce, ...",...,Preheat oven to 300 degrees F (150 degrees C)....,Preheat oven to 300 degrees F (150 degrees C)....,Preheat oven to 375 degrees F (190 degrees C)....,Pour cream into a blender. Cover and blend unt...,Saute onions in olive oil over medium heat unt...,Heat oil in a large pot over medium heat. Add ...,Heat cream and coconut extract in a skillet or...,Heat bacon grease in a skillet over medium-hig...,Preheat oven to 350 degrees F (175 degrees C)....,Heat 1/4 cup canola oil in large skillet over ...
picture_link,55lznCYBbs2mT8BTx6BTkLhynGHzM.S,QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,YCnbhplMgiraW4rUXcybgSEZinSgljm,jRnWGDXDdyOg3rta4/HVAR2rD19XubC,aUca10AaD8T2yYvcLOgH/UJlR5/OhOe,YdgEVyLVffZgh9NZPN3Eqj6MaX8KdzK,UrgvDGu4roLiho160fTVIwCUrGZna8i,OFp6yXFwzlrkMQ5STffYPllxQvMVLUS,...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,cWEzUSv9Ozr3b4MxNVCqJYgTjIS.kHm,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa


In [179]:
# For the moment we are not interested in the instructions or picture_link
df.drop(["instructions", "picture_link"], inplace=True)
df.head()

,rmK12Uau.ntP510KeImX506H6Mr6jTu,5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,clyYQv.CplpwJtjNaFGhx0VilNYqRxu,BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,kq.naD.8G19M4UU9dVvJgHtpfo.l/eC,lYrgWNn00EXblOupzM3tL0jGr9O0CB2,Fu0DgGYFUGwc0BBlN6r20o/ihOVs5bO,MBRNtqELRRuv8zJH4k7Aba2bmIc2A3C,ZPyPoMiNvgAfrKcRpH9FEYV/XsPZBsW,...,uHHb42/tuIKsmN5U6l9AD.FdVpSFxs6,IPAeN3L6rm2oughJpUhbG038k.ACJ0K,3UIhlTQFH5jyIaHN8zeKlK5V.94Kjwu,PdBxkE2gnI/.ynokkp1Hu1KLGZnGdei,SB46Udqc5Svsi70S1qRmRLv5tlg8Oca,gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
title,Slow Cooker Chicken and Dumplings,Awesome Slow Cooker Pot Roast,Brown Sugar Meatloaf,Best Chocolate Chip Cookies,Homemade Mac and Cheese Casserole,Banana Banana Bread,Chef John's Fisherman's Pie,Mom's Zucchini Bread,The Best Rolled Sugar Cookies,Singapore Chili Crabs,...,Citrus Creme Brulee,Honey Nutty Granola,Salmon en Croute,Homemade Blender Butter,Gluten Free Chicken Noodle Soup,Thai-Indian Veggie Soup,Coconut Milk-Free Panang Curry Chicken,Cooked Cold Salad,Easy Eggnog Creme Brulee,Super Power Stovetop Granola
ingredients,"[4 skinless, boneless chicken breast halves, 2...",[2 (10.75 ounce) cans condensed cream of mushr...,"[1/2 cup packed brown sugar, 1/2 cup ketchup, ...","[1 cup butter, softened, 1 cup white sugar, 1 ...","[8 ounces whole wheat rotini pasta, 3 cups fre...","[2 cups all-purpose flour, 1 teaspoon baking s...","[For potato crust:, 3 russet potatoes, peeled ...","[3 cups all-purpose flour, 1 teaspoon salt, 1 ...","[1 1/2 cups butter, softened, 2 cups white sug...","[Sauce:, 1/2 cup ketchup, 1/2 cup chicken brot...",...,"[2 oranges, juiced, lemon, juiced, 1 lime, jui...","[3 cups rolled oats, 1 1/2 cups toasted almond...","[1 cup watercress, or as desired, 1/2 lemon, z...","[2 pints heavy whipping cream, salt to taste]",[1/2 (12 ounce) box Barilla® Gluten Free Elbow...,"[2 teaspoons olive oil, 1/4 cup minced fresh g...","[2 cups light cream, 1/4 teaspoon coconut extr...","[3 tablespoons bacon grease, 2 cups shredded B...","[4 egg yolks, 1 tablespoon white sugar, 1 cup ...","[1/4 cup canola oil, 3 cups quick-cooking oats..."


In [180]:
# Select a subset of data to look at, we transpose the original data frame
# so we can select all the values in a column at once.
df_trans = df.transpose()
# drop the recipes that have NaN for their ingredients
df_trans = df_trans[df_trans.ingredients.notnull()]

In [181]:
df_trans.head()

,title,ingredients
rmK12Uau.ntP510KeImX506H6Mr6jTu,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves, 2..."
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar, 1/2 cup ketchup, ..."
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,Best Chocolate Chip Cookies,"[1 cup butter, softened, 1 cup white sugar, 1 ..."
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,Homemade Mac and Cheese Casserole,"[8 ounces whole wheat rotini pasta, 3 cups fre..."


In [182]:
import utils
tagger = pycrfsuite.Tagger()
tagger.open('trained_pycrfsuite')

nlp_engine = spacy.lang.en.English()

from nltk.tokenize import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()

def sent2labels(sent):
    return [word[-1] for word in sent]

def sent2features(sent):
    return [word[:-1] for word in sent]

def sent2tokens(sent):
    return [word[0] for word in sent]   

def get_sentence_features(sent):
    """Gets  the features of the sentence"""
    sent_tokens = utils.tokenize(utils.cleanUnicodeFractions(sent))

    sent_features = []
    for i, token in enumerate(sent_tokens):
        token_features = [token]
        token_features.extend(utils.getFeatures(token, i+1, sent_tokens))
        sent_features.append(token_features)
    return sent_features

def format_ingredient_output(tagger_output, display=False):
    """Formats the tagger output into a more convenient dictionary"""
    data = [{}]
    display = [[]]
    prevTag = None

    
    for token, tag in tagger_output:
    # turn B-NAME/123 back into "name"
        tag = re.sub(r'^[BI]\-', "", tag).lower()
        # ---- DISPLAY ----
        # build a structure which groups each token by its tag, so we can
        # rebuild the original display name later.

        if prevTag != tag:
            display[-1].append((tag, [token]))
            prevTag = tag
        else:
            display[-1][-1][1].append(token)
            #               ^- token
            #            ^---- tag
            #        ^-------- ingredient

            # ---- DATA ----
            # build a dict grouping tokens by their tag

            # initialize this attribute if this is the first token of its kind
        if tag not in data[-1]:
            data[-1][tag] = []

        # HACK: If this token is a unit, singularize it so Scoop accepts it.
        if tag == "unit":
            token = utils.singularize(token)

        data[-1][tag].append(token)

    # reassemble the output into a list of dicts.
    output = [
        dict([(k, utils.smartJoin(tokens)) for k, tokens in ingredient.items()])
        for ingredient in data
        if len(ingredient)
    ]
    
    # Add the raw ingredient phrase
    for i, v in enumerate(output):
        output[i]["input"] = utils.smartJoin(
            [" ".join(tokens) for k, tokens in display[i]])

    return output

def parse_ingredient(sent):
    """ingredient parsing logic"""
    sentence_features = get_sentence_features(sent)
    tags = tagger.tag(sentence_features)
    tagger_output = zip(sent2tokens(sentence_features), tags)
    parsed_ingredient =  format_ingredient_output(tagger_output)
    if parsed_ingredient:
        parsed_ingredient[0]['name'] = parsed_ingredient[0].get('name','').strip('.')
    return parsed_ingredient

In [183]:
def parse_recipe_ingredients(ingredient_list):
    """Wrapper around parse_ingredient so we can call it on an ingredient list"""
    #sentences = tokenizer.tokenize(q)
    sentences = tokenizer.tokenize(ingredient_list)
    sentences = [sent.strip('\n') for sent in sentences]
    ingredients = []
    for sent in sentences:
        ingredients.extend(parse_ingredient(sent))
    return ingredients

def is_seperator(ingredient_string):
    return (
        ingredient_string.find("For ") == 0
        or " " not in ingredient_string
        or (":" in ingredient_string and "eg:" not in ingredient_string)
    )

In [157]:
s = "2 (10.75 ounce) cans condensed cream of mushr"
pattern = re.compile(r'([0-9]+)\s\(([0-9\.\sa-z]+)\)\s(\w+)')
match = re.sub(pattern, lambda x: str(int(x.group(1))*float(x.group(2).split()[0])) + " " + x.group(2).split()[1], s)
print(match)

21.5 ounce condensed cream of mushr


In [202]:

pattern1 = re.compile(r'([0-9]+)\s\(([0-9\.]+\s[a-z]+)\)\s(\w+)')
pattern2 = re.compile(r'^([0-9]+)\s\(*([0-9\.\s\/]+)[\s\-](inch)[\sa-z]*\)*')

def parse_list(recipe_index, recipe_title, ingredient_list):
    if isinstance(ingredient_list, list):
        ingredients=[]
        for ingredient_string in ingredient_list:
            # check if not ingredient, but separator
            # e.g. "For Bread:"
            if is_seperator(ingredient_string):
                continue
                
            if "skewers" in ingredient_string:
                continue
                
            if "canning jars" in ingredient_string:
                continue
                
            # remove trademark symbols
            ingredient_string = ingredient_string.replace("\u00ae", "")
            ingredient_string = ingredient_string.replace("(TM)", "")
            ingredient_string = ingredient_string.replace("™", "")
            
            ingredient_string = re.sub(pattern1, lambda x: str(int(x.group(1))*float(x.group(2).split()[0])) + " " + x.group(2).split()[1], ingredient_string)
            match = re.search(pattern2, ingredient_string)
            if match:
                try:
                    qty = float(match.group(1))
                    mult = match.group(2).split()
                    if len(mult) == 2:
                        qty *= float(mult[0]) + eval(mult[1])
                    else:
                        qty *= eval(mult[0])
                    ingredient_string = re.sub(pattern2, str(qty) + " " + "inch", ingredient_string)
                except ValueError:
                    pass
            
            ingredient = {}

            #ingr = ingre_text[2]
            #print(ingr)
            ingr_parse = parse_recipe_ingredients(ingredient_string)
            #print(ingr_parse)
            ingredient = ingr_parse[0]
            ingredient["title"] = recipe_title
            ingredient["index"] = recipe_index
            if "qty" not in ingredient and "other" in ingredient:
                try:
                    ingredient["qty"] = float(ingredient["other"])
                    ingredient["other"] = np.nan
                except ValueError:
                    pass
            if "qty" in ingredient:
                if not isinstance(ingredient["qty"], float):
                    if "$" in ingredient["qty"]:
                        try:
                            frac = ingredient["qty"].split("$")
                            ingredient["qty"] = float(frac[0])+eval(frac[1])
                        except (ValueError, NameError):
                            pass
                    else:
                        try:
                            ingredient["qty"] = float(ingredient["qty"])
                        except ValueError:
                            ingredient["qty"] = np.nan
                            
                        

            ingredients.append(ingredient)
        return ingredients
    else:
        return recipe_index, recipe_title, None

In [203]:
vfunc = np.vectorize(parse_list)

ingredient_df = pd.DataFrame(
    list(
        chain.from_iterable(
            vfunc(
                df_trans.index.values,
                df_trans.title.values,
                df_trans["ingredients"].values,
            )
        )
    )
)
ingredient_df.set_index("index", inplace=True)

In [204]:

ingredient_df.head(10)


,qty,comment,name,input,title,unit,other,range_end
index,,,,,,,,
rmK12Uau.ntP510KeImX506H6Mr6jTu,4.00,"skinless, boneless",chicken breast halves,"4 skinless, boneless chicken breast halves",Slow Cooker Chicken and Dumplings,NaN,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,2.00,NaN,butter,2 tablespoons butter,Slow Cooker Chicken and Dumplings,tablespoon,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,21.50,NaN,condensed cream of chicken soup,21.5 ounce condensed cream of chicken soup,Slow Cooker Chicken and Dumplings,ounce,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,1.00,finely diced,onion,"1 onion, finely diced",Slow Cooker Chicken and Dumplings,NaN,",",NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,20.00,", torn into pieces",refrigerated biscuit dough,"20.0 ounce refrigerated biscuit dough, torn in...",Slow Cooker Chicken and Dumplings,ounce,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,21.50,NaN,condensed cream of mushroom soup,21.5 ounce condensed cream of mushroom soup,Awesome Slow Cooker Pot Roast,ounce,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,1.00,NaN,dry onion soup mix,1.0 ounce dry onion soup mix,Awesome Slow Cooker Pot Roast,ounce,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,1.25,NaN,water,1$1/4 cups water,Awesome Slow Cooker Pot Roast,cup,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,5.50,NaN,pot roast,5$1/2 pounds pot roast,Awesome Slow Cooker Pot Roast,pound,NaN,NaN


In [205]:
ingredient_df.qty.unique()

array([4.00000000e+00, 2.00000000e+00, 2.15000000e+01, 1.00000000e+00,
       2.00000000e+01, 1.25000000e+00, 5.50000000e+00,            nan,
       1.50000000e+00, 3.00000000e+00, 8.00000000e+00, 2.50000000e+00,
       1.20000000e+01, 2.25000000e+00, 5.00000000e+00, 6.00000000e+00,
       1.50000000e+01, 3.50000000e+00, 4.50000000e+00, 6.80000000e+00,
       2.80000000e+01, 1.30000000e+01, 1.60000000e+01, 3.25000000e+00,
       1.75000000e+00, 1.80000000e+01, 3.96000000e+01, 3.20000000e+01,
       4.50000000e+01, 5.60000000e+01, 1.05000000e+01, 1.07500000e+01,
       2.75000000e+00, 5.20000000e+01, 1.82500000e+01, 5.90000000e+00,
       1.00000000e+01, 1.45000000e+01, 7.00000000e+00, 7.20000000e+01,
       7.00000000e-01, 4.00000000e+01, 9.60000000e+01, 9.00000000e+00,
       8.50000000e+00, 1.47500000e+01, 1.40000000e+01, 2.50000000e-01,
       4.35000000e+01, 3.75000000e+00, 2.90000000e+01, 8.00000000e+01,
       5.80000000e+01, 2.40000000e+01, 6.00000000e+01, 2.70000000e+01,
      

In [206]:
# Now we convert as many units as possible to metric
ingredient_df.loc[ingredient_df.unit == "pound", "qty"] *= 453.592
ingredient_df.loc[ingredient_df.unit == "pound", "unit"] = "grams"

ingredient_df.loc[ingredient_df.unit == "teaspoon", "qty"] *= 4.92892
ingredient_df.loc[ingredient_df.unit == "teaspoon", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "tablespoon", "qty"] *= 14.7868
ingredient_df.loc[ingredient_df.unit == "tablespoon", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "cup", "qty"] *= 236.588
ingredient_df.loc[ingredient_df.unit == "cup", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "pinch", "qty"] *= 4.92892 * (1 / 16)
ingredient_df.loc[ingredient_df.unit == "pinch", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "dash", "qty"] *= 4.92892 * (1 / 8)
ingredient_df.loc[ingredient_df.unit == "dash", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "ounce", "qty"] *= 28.3495
ingredient_df.loc[ingredient_df.unit == "ounce", "unit"] = "grams"

ingredient_df.loc[ingredient_df.unit == "fluid ounce", "qty"] *= 29.5735
ingredient_df.loc[ingredient_df.unit == "fluid ounce", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "pint", "qty"] *= 473.176
ingredient_df.loc[ingredient_df.unit == "pint", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "quart", "qty"] *= 946.353
ingredient_df.loc[ingredient_df.unit == "quart", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "liter", "qty"] *= 1000
ingredient_df.loc[ingredient_df.unit == "liter", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "gallon", "qty"] *= 3785.41
ingredient_df.loc[ingredient_df.unit == "gallon", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "drop", "qty"] *= 0.05
ingredient_df.loc[ingredient_df.unit == "drop", "unit"] = "milliliters"

ingredient_df.loc[ingredient_df.unit == "jigger", "qty"] *= 44.3603
ingredient_df.loc[ingredient_df.unit == "jigger", "unit"] = "milliliters"

ingredient_df.head(10)

,qty,comment,name,input,title,unit,other,range_end
index,,,,,,,,
rmK12Uau.ntP510KeImX506H6Mr6jTu,4.00000,"skinless, boneless",chicken breast halves,"4 skinless, boneless chicken breast halves",Slow Cooker Chicken and Dumplings,NaN,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,29.57360,NaN,butter,2 tablespoons butter,Slow Cooker Chicken and Dumplings,milliliters,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,609.51425,NaN,condensed cream of chicken soup,21.5 ounce condensed cream of chicken soup,Slow Cooker Chicken and Dumplings,grams,NaN,NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,1.00000,finely diced,onion,"1 onion, finely diced",Slow Cooker Chicken and Dumplings,NaN,",",NaN
rmK12Uau.ntP510KeImX506H6Mr6jTu,566.99000,", torn into pieces",refrigerated biscuit dough,"20.0 ounce refrigerated biscuit dough, torn in...",Slow Cooker Chicken and Dumplings,grams,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,609.51425,NaN,condensed cream of mushroom soup,21.5 ounce condensed cream of mushroom soup,Awesome Slow Cooker Pot Roast,grams,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,28.34950,NaN,dry onion soup mix,1.0 ounce dry onion soup mix,Awesome Slow Cooker Pot Roast,grams,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,295.73500,NaN,water,1$1/4 cups water,Awesome Slow Cooker Pot Roast,milliliters,NaN,NaN
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,2494.75600,NaN,pot roast,5$1/2 pounds pot roast,Awesome Slow Cooker Pot Roast,grams,NaN,NaN


In [209]:
# Now we can explore the results of our parsing
print("Unique units of measurement:\n", ingredient_df.unit.unique())
print("Number of unique ingredients: ", len(ingredient_df.name.unique()))
ingredient_df.name.unique()
temp_df = ingredient_df[ingredient_df.name.str.contains("honey")]
temp_df.name.unique()

Unique units of measurement:
 [nan 'milliliters' 'grams' 'clove' 'inch' 'head' 'bunch' 'slice' 'stalk'
 'cube' 'packet' 'milliliter' 'sprig' 'ear' 'fillet' 'loaf' 'strip' 'bulb'
 'pound fillet' 'fluid' 'envelope' 'leaf' 'ounce slice' 'box' 'rack'
 'bunch stalk' 'inch slice' 'stick' 'ounce fillet' 'clove clove'
 'ounce pound' 'sheet' 'splash' 'piece' 'bulb clove' 'ball' 'package'
 'wedge' 'gram' 'scoop' 'pound slice' 'head clove' 'stalk stalk' 'bag'
 'cup clove' 'cup teaspoon' 'cup slice' 'square' 'bone' 'cup fillet'
 'bottle' 'link' 'jar' 'head bulb' 'inch sprig' 'chunk' 'cup cup']
Number of unique ingredients:  13004


array(['honey', 'honey mustard salad dressing', 'dijon honey mustard',
       'honey mustard', 'honey barbeque sauce',
       'honey-roasted sliced almonds', 'honey barbecue sauce',
       'clover honey', 'soft cream cheese with honey and nuts',
       'orange blossom honey', 'avocado honey', 'honey graham cereal',
       'honey ham', 'honey mustard barbecue sauce', 'raw honey',
       'honey and nut flavor cornflakes cereal',
       'honey baked ham, cut into 1/2 inch cubes', 'wildflower honey',
       'honey mustard salad dressing mix', 'honey garlic sauce',
       'honey teriyaki sauce', 'honey flavored bacon', 'honeycomb tripe',
       'honeydew melon', 'honey mustard dressing',
       'honey Dijon salad dressing', 'honey mustard and onion pretzels',
       'honey (fireweed honey preferred)', 'thyme honey', 'honeydew',
       'honey garlic barbecue sauce', 'honeydew melon balls',
       'honey roasted peanuts', 'agave nectar or honey',
       'honey-flavored barbeque sauce', 'honey

In [211]:
# Now we make the real DataFrame that we will do most of our work with.
# In theory this should have worked with .pivot instead of .pivot_table,
# but it didn't. This makes me think there is a bug somewhere in how
# the recipes are being parsed but I will look into that later
# FYI this will take several minutes
recipe_df = ingredient_df.pivot_table(
    index="index", columns="name", values="qty", aggfunc=np.mean
)
recipe_df.fillna(0, inplace=True)
recipe_df.head(10)

name,,'bouqet garni' spice balls,(.68,(0.5 oz each) Cheddar cheese,(1 inch thick) beef top sirloin steak,(1 inch thick) steak fillet,(1$1/2-pound) boneless pork loin,(1$1/2-pound) boneless pork loin roast,(1$1/2-pound) flank steak,(1$1/2-pound) trimmed beef flank steak,...,zucchini matchsticks,zucchini or yellow summer squash,zucchini slices,zucchini squash,zucchini squashes,"zucchini, quartered lengthwise and then cut crosswise into 1/2-inch pieces","zucchini, sliced into long ribbons using a vegetable peeler",zucchinis,zwieback cracker crumbs,zwieback toast
index,,,,,,,,,,,,,,,,,,,,,
../AFIrLizRK7C5isREQ0hUdXoNXTpS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..3M3xsVPQLnncQ2oYzCyhGjNe/6EB2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..64QbeV22IrPuRAdLu4zscfAjsaFaK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..6gl/0GIzSugrbvy6kHyxEHMMP4h9K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..B3IS4KrEGWH7o13lvV9nQSe9a4u36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..G2iuRLT7oseNmkKbjqAgbdB1G5IBC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..M776r8BHroHcCXw3vInfs25LDkWw2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..VDFobAHiAa0127h.N784Y2pnZyBZK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
..hYtxdDoy5Rp..pLgIj2mKKfvp1jPa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Let's save our dataframe so we can look at it without having to reload and recompute everything later.
# pickle is much faster than csv
recipe_df.to_pickle("data/"+file_name+"_parsed.pickle")

In [ ]:
# Here we compute the t-SNE
# WARNING: THIS TAKES A LONG TIME FOR LARGE DATAFRAMES
data = recipe_df[recipe_df.columns].values
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=5000)
tsne_results = tsne.fit_transform(data)
print("tsne done")
recipe_df["tsne-2d-one"] = tsne_results[:, 0]
recipe_df["tsne-2d-two"] = tsne_results[:, 1]

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 39416 samples in 91.124s...


In [ ]:
# plot the t-SNE
plt.figure(figsize=(16, 10))
sns.scatterplot(x="tsne-2d-one", y="tsne-2d-two", data=recipe_df, linewidth=0)
plt.show()

In [27]:
# Here I'm just exploring outliers in the t-SNE
sample_df = recipe_df.loc[recipe_df["tsne-2d-two"] <= -100]
drop_cols = sample_df.columns[(sample_df == 0).sum() == sample_df.shape[0]]
sample_df.drop(drop_cols, axis=1, inplace=True)
sample_df.columns.unique()


Index(['Golden Delicious apple', 'Italian bread',
       'Mexican cheese blend or more to taste', 'Parmesan cheese',
       'Romano cheese', 'all purpose apples', 'all purpose flour',
       'apple cider vinegar', 'baby back ribs', 'baking powder',
       ...
       'watermelon', 'whipped topping', 'white bread', 'white bread cubes',
       'white peaches', 'white sugar', 'white wine', 'whole wheat bread',
       'tsne-2d-one', 'tsne-2d-two'],
      dtype='object', name='ingredient', length=118)